In [ ]:
# Install TensorFlow and other required libraries
!pip install tensorflow

# Import necessary libraries
import tensorflow as tf
import numpy as np


In [ ]:
pip install emoji==1.7

In [ ]:
pip install spellchecker

In [ ]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import argparse
import io
import json
import os
import re
import sklearn
import sys

# Please use python 3.5 or above
import numpy as np
import tensorflow as tf
from emoji import UNICODE_EMOJI
from keras import optimizers
from keras.backend import argmax
from keras.layers import Dense, Embedding, LSTM, LeakyReLU, Concatenate, Input, Bidirectional, Conv1D, MaxPooling1D, \
    Reshape, MaxPooling2D, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras.metrics import categorical_accuracy, binary_accuracy
from keras.models import Sequential, Model
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
from spellchecker import SpellChecker



In [ ]:
pip install pyspellchecker

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# Define the dataset name
dataset_name = "emo"

# Load the dataset
dataset = load_dataset(dataset_name)

# Print some information about the dataset
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 30160
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5509
    })
})


In [ ]:
from datasets import load_dataset

# Define the dataset name
dataset_name = "emo"

# Load the dataset
dataset = load_dataset(dataset_name)

# Save the dataset to a directory in your Colab environment
save_path = "/content/dataset"  # Change this to your desired save path
dataset.save_to_disk(save_path)


Saving the dataset (0/1 shards):   0%|          | 0/30160 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5509 [00:00<?, ? examples/s]

In [ ]:
def clean_text(text):
    # Remove special characters and punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize (split the text into words)
    tokens = text.split()
    # Join tokens back into a string
    cleaned_text = " ".join(tokens)
    return cleaned_text
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# Clean and preprocess the training data
cleaned_train_data = [clean_text(example["text"]) for example in train_dataset]

# Clean and preprocess the test data
cleaned_test_data = [clean_text(example["text"]) for example in test_dataset]

# Print the cleaned data (first few examples)
print("Cleaned Training Data (First 5 examples):")
for i in range(5):
    print(cleaned_train_data[i])

print("\nCleaned Test Data (First 5 examples):")
for i in range(5):
    print(cleaned_test_data[i])

Cleaned Training Data (First 5 examples):
don t worry i m girl hmm how do i know if you are what s ur name
when did i saw many times i think no i never saw you
by by google chrome where you live
u r ridiculous i might be ridiculous but i am telling the truth u little disgusting whore
just for time pass wt do u do 4 a living then maybe

Cleaned Test Data (First 5 examples):
hmm what does your bio mean i dont have any bio
what you like very little things ok
yes how so i want to fuck babu
what did you guess what what fuck
we of course we will what gender movies you like


In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from datasets import load_dataset

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:

def preprocess_text(text):
    # Remove special characters and punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize (split the text into words)
    tokens = text.split()

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back into a string
    preprocessed_text = " ".join(tokens)
    return preprocessed_text

# Preprocess the training data
preprocessed_train_data = [preprocess_text(example["text"]) for example in train_dataset]

# Preprocess the test data
preprocessed_test_data = [preprocess_text(example["text"]) for example in test_dataset]

# Print the preprocessed data (first few examples)
print("Preprocessed Training Data (First 5 examples):")
for i in range(5):
    print(preprocessed_train_data[i])

print("\nPreprocessed Test Data (First 5 examples):")
for i in range(5):
    print(preprocessed_test_data[i])

Preprocessed Training Data (First 5 examples):
don t worry i m girl hmm how do i know if you are what s ur name
when did i saw many time i think no i never saw you
by by google chrome where you live
u r ridiculous i might be ridiculous but i am telling the truth u little disgusting whore
just for time pas wt do u do 4 a living then maybe

Preprocessed Test Data (First 5 examples):
hmm what doe your bio mean i dont have any bio
what you like very little thing ok
yes how so i want to fuck babu
what did you guess what what fuck
we of course we will what gender movie you like


In [ ]:
MAX_NB_WORDS = 5000  # Maximum number of unique words in the vocabulary
MAX_SEQUENCE_LENGTH = 100  # Maximum length of a text sequence
EMBEDDING_DIM = 50  # Dimension of word embeddings
NUM_CLASSES = 4  # Number of classes in the dataset
BATCH_SIZE = 64  # Batch size for training
NUM_EPOCHS = 10  # Number of training epochs

In [2]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(preprocessed_train_data)
train_sequences = tokenizer.texts_to_sequences(preprocessed_train_data)
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Tokenize and pad the test data
test_sequences = tokenizer.texts_to_sequences(preprocessed_test_data)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Convert labels to one-hot encoding
train_labels = to_categorical(np.asarray(train_dataset["label"]), num_classes=NUM_CLASSES)

# Build an embedding matrix
word_index = tokenizer.word_index
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))

# Load pre-trained word embeddings (e.g., GloVe embeddings)
# You can replace this with your path to the word embeddings file
glove_path = "glove.txt"
embeddings_index = {}
with open(glove_path, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Build and train a model
model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(NUM_CLASSES, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(train_data, train_labels, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

# Make predictions on the test data
test_predictions = model.predict(test_data)


NameError: ignored

In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Ground truth labels (true values) from the test dataset
true_labels = test_dataset["label"]

# Predicted labels from the model
predicted_labels = np.argmax(test_predictions, axis=1)  # Convert one-hot to categorical labels

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# Calculate precision, recall, and F1-score
precision = precision_score(true_labels, predicted_labels, average="weighted")
recall = recall_score(true_labels, predicted_labels, average="weighted")
f1 = f1_score(true_labels, predicted_labels, average="weighted")

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Print classification report
class_names = ["Class 0", "Class 1", "Class 2", "Class 3"]  # Adjust class names based on your dataset
report = classification_report(true_labels, predicted_labels, target_names=class_names)
print("Classification Report:\n", report)

# Generate a confusion matrix
confusion = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:\n", confusion)


NameError: ignored